<a href="https://colab.research.google.com/github/idmdvan/letstrythis/blob/master/SDA_Montreal_with_improved_SA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Algoritmo de diseño de servicios
##Autor: Iván Daniel Moreno Díaz

In [ ]:
#Original file is located at
#https://colab.research.google.com/drive/1PEs7RlL_ZhBPw25S-GX_1yxRabhDHSQI


# Parámetros iniciales

In [22]:
#Define el directorio donde están los archivos

import sys
sys.path.insert(0,'/content/drive/MyDrive/Doctorado/Tesis/Coding/')
line_file = '/content/drive/MyDrive/Doctorado/Tesis/Coding/Montréal/lines_67.csv'
demand_file = '/content/drive/MyDrive/Doctorado/Tesis/Coding/Montréal/Demanda_67.csv'
distance_file = '/content/drive/MyDrive/Doctorado/Tesis/Coding/Montréal/distancias_67.csv'
fmin = 4 #frecuencia mínima en bus/h
flota = 100 #flota disponible
punto_inicial = 'NOC' #modo de repartir la flota disponible
t_input = 1000000 #temperatura inicial
#NOC = No change, deja la frecuencia tal como viene en el .txt
#MIN = asigna a todos la flota para cumplir la frecuencia mínima
#REP = repartir la flota disponible entre todos los servicios, Esto no garantiza que haya balance de frecuencias
#SIN = Comenzar sin servicios expresos. 
        #esto es, signar la frecuencia mínima a los servicios expresos. 
        #El resto de la flota para el all-stop (servicios número 1 y 2 de la lista)
#FRR: todos los servicios comienzan con 10 bus/h
all_stop_ida = '67N'
all_stop_reg = '67S'
det = 1/60 #tiempo de detencion en horas. ojo que non sirve, cambiar "p"
vst_veh = 7 #CAD/h
vst_esp = 2*vst_veh
vst_trb = 4*vst_veh
vel = 20 #km/h
costo_dist = 4 #CAD/km
costo_tpo = 20 #CAD/h
cap = 180

In [23]:
"""Importa librerías"""
#!pip install geopandas
import pandas as pd
import networkx as nx
import csv
import operator
import matplotlib.pyplot as plt
import time
import math
import numpy as np
import itertools
import statistics
import random as rand
import matplotlib
#import geopandas 

# Asignación

In [24]:
"""
**leer**: Lee distancias y estaciones. Crea grafo B
"""

def leer(dist=distance_file):
    #se leen las estaciones de la red
    t_viaje_a={} #matriz con tiempos de viaje en segmentos de linea (grafo A)
    estaciones=[] #lista de estaciones
    with open(dist,'r') as f:
        reader = csv.reader(f, delimiter = ",")
        for row in reader:
            sentido=row[3]
            t_viaje_a[str(row[0]),str(row[1])]=float(row[5])
            if row[0] not in estaciones:
                estaciones.append(str(row[0]))
            if row[1] not in estaciones or row[1] not in estaciones:
                estaciones.append(str(row[1]))
            '''
            if sentido=='i':
                sentido='r'
                #t_viaje_a[row[1]+sentido,row[0]+sentido]=float(row[5]) #simetrica
                t_viaje_a[row[1],row[0]]=float(row[5]) #simetrica
                del sentido
            else:
                sentido='i'
                #t_viaje_a[row[1]+sentido,row[0]+sentido]=float(row[5])#simetrica
                t_viaje_a[row[1],row[0]]=float(row[5])
                del sentido
            '''
    #se crea el grafo B, que contiene los arcos físicos de la red
    
    B=nx.DiGraph() 
    for i in t_viaje_a:
        origen=i[0]
        destino=i[1]
        distancia=float(t_viaje_a[i])
        B.add_edge(str(origen),str(destino),weight=distancia)
    '''
    for i in estaciones:
        if i[-1]=='i':
            j=i[:-1]+'r'
            B.add_edge(i,j,weight=0)
        if i[-1]=='r':
            j=i[:-1]+'i'
            B.add_edge(i,j,weight=0)
    '''
    return t_viaje_a, estaciones, B

"""**crear**: Crea los grafos A y L. L contiene secciones de ruta para cada servicio, y ambos grafos tienen arcos ficticios de trasbordo. Se almacena información de capacidad de los buses, frecuencia, tiempos de viaje"""

def crear (B, dist = distance_file, lineas =line_file, demanda = demand_file):
    k = 1 #parámetro de regularidad
    p = 1/60 #penalización por trasbordo
    d = 1/60 #tiempo de detención
    A = nx.DiGraph()
    L = nx.DiGraph()
    frecuencia, capacidad, recorrido = {}, {}, {}
    #se lee el recorrido de cada servicio    
    with open(lineas, 'r') as g:
        reader = csv.reader(g, delimiter = ',')
        for row in reader:
            #print(row)
            frecuencia[row[0]] = row[1]
            capacidad[row[0]] = row[2]
            secuencia = []
            for parada in range(3,len(row)):
                parada_actual = (str(row[parada]))
                if parada_actual[-2:] == '.0':
                  parada_actual = parada_actual[:-2]
                if len(parada_actual) > 0:
                    secuencia.append(parada_actual)
                recorrido[row[0]] = secuencia
    #se el grafo L de secciones de ruta, a partir del recorrido de cada servicio
    for linea in recorrido:
        for parada in range(len(recorrido[linea])-1):
            origen = recorrido[linea][parada]
            pos_origen = recorrido[linea].index(origen)
            it = 1
            while pos_origen + it < len(recorrido[linea]):
                destino = recorrido[linea][pos_origen+it]
                nombre = (origen,destino)
                if (origen,destino) not in A.edges() and it==1:
                    A.add_edge(origen,destino,key=nombre)
                if (origen,destino) not in L.edges():
                    L.add_edge(origen,destino,key=nombre)
                it = it + 1
    '''
    #añadir arcos ficticios de combinación (xi-xr)
    for i in A.nodes():
        if i[-1] == 'i':
            est = i[0:-1]+'i'
            estr= i[0:-1]+'r'
            #costo de transferencia  = 0
            A.add_edge(est,estr,weight=0)
            A.add_edge(estr,est,weight=0)
            L.add_edge(est,estr,weight=0)
            L.add_edge(estr,est,weight=0)
            serfict_i = i[0:-1]+'ir'
            serfict_r = i[0:-1]+'ri'
            recorrido[serfict_i]=[i[0:-1]+'i',i[0:-1]+'r']
            recorrido[serfict_r]=[i[0:-1]+'r',i[0:-1]+'i']  
            frecuencia[serfict_i]='1000' #frecuencia infinita para el trasbordo
            frecuencia[serfict_r]='1000'
    '''
    #se estima para cada sección, su tiempo de viaje y de detención
    segmento, lineas_comunes, ttv, tiempo_de_viaje = {}, {}, {}, {}
    for edge in L.edges():
        segmento[edge] = {}
        lineas_comunes[edge] = []
        i,j = edge[0],edge[1]
        if i[0:-1] != j[0:-1]:
            for linea in recorrido:
                if i in recorrido[linea]:
                    if j in recorrido[linea]:
                        #formato: (tiempo en horas, frecuencia en bus/h)
                        tiempo_de_viaje[(i,j)] = nx.shortest_path_length(B,source = i, target = j, weight='weight')
                        num_est = recorrido[linea].index(j)-recorrido[linea].index(i)
                        num_est = num_est - 1
                        t_via = float(tiempo_de_viaje[(i,j)]) + p + (num_est*d)
                        segmento[edge][linea] = (t_via,frecuencia[linea])
    return k, p, d, A, L, frecuencia, capacidad, recorrido, segmento, lineas_comunes, ttv, tiempo_de_viaje

In [25]:
"""**chriquiB** Encuentra el conjunto de líneas comunes para cada sección de ruta"""

def chriquiB (frecuencia, tiempo_de_viaje, L, ttv, segmento, lineas_comunes, k, p, d):
    frec_l_com = {}
    for edge in L.edges():
        if edge[0][0:-1] != edge[1][0:-1]:
            segmento_tw={}
            for linea in segmento[edge]:
                tiempo = segmento[edge][linea][0] + (k/float(segmento[edge][linea][1]))
                segmento_tw[linea] = tiempo
            sorted_lines = sorted(segmento_tw, key = segmento_tw.__getitem__)
            lin_min = sorted_lines[0]
            lineas_comunes[edge].append(lin_min)
            ttv[edge] = segmento[edge][lin_min][0] + (k/float(frecuencia[lin_min])) - p
            if len(sorted_lines) > 1:
                for linea in range(1,len(sorted_lines)):
                    candidato = sorted_lines[linea]
                    lineas_comunes[edge].append(candidato)
                    num_hip = k
                    den_hip = 0
                    for i in range(len(lineas_comunes[edge])):
                        num_hip = num_hip + float(frecuencia[lineas_comunes[edge][i]])*((segmento[edge][lineas_comunes[edge][i]][0])-p)
                        den_hip = den_hip + float(frecuencia[lineas_comunes[edge][i]])
                    ttv_hip = num_hip / den_hip
                    if ttv_hip > ttv[edge]:
                        lineas_comunes[edge].pop()
                        break
                    elif ttv_hip < ttv[edge]:
                        ttv[edge] = ttv_hip
        '''
        elif edge[0][0:-1]==edge[1][0:-1] and edge[0][-1]=='i' and edge[1][-1]=='r':
            lineas_comunes[edge]=[edge[0][0:-1]+'ir']
            segmento[edge][edge[0][0:-1]+'ir']=(0.000001,1000) ##tiempo de trasbordo de 3 min y frec 100
            frec_l_com[edge]='1000'
            ttv[edge]=0.00001
        elif edge[0][0:-1]==edge[1][0:-1] and edge[0][-1]=='r' and edge[1][-1]=='i':
            lineas_comunes[edge]=[edge[0][0:-1]+'ri']
            segmento[edge][edge[0][0:-1]+'ri']=(0.000001,1000)##tiempo de trasbordo de 3 min y frec 100
            frec_l_com[edge]='1000'
            ttv[edge]=0.00001
        '''
    for par in lineas_comunes:
        fefec = 0
        for linea in range(len(lineas_comunes[par])):
            fefec = fefec + float(frecuencia[lineas_comunes[par][linea]])
            frec_l_com[par] = fefec
    for edge in L.edges():
        if edge[0][0:-1]!=edge[1][0:-1]:
            L.edges[edge]['weight']=ttv[edge]
        else:
            L.edges[edge]['weight']=0
    return frec_l_com, ttv, lineas_comunes

In [26]:
"""**Costos** Calcula todas las componentes del costo del sistema, además de los perfiles de carga. Asigna la demanda a los servicios disponibles considerando el conjunto de líneas comunes. """

def Costos (A,B,L,pen_ex,k,p,d,ttv,frec_l_com,frecuencia,lineas_comunes,recorrido,segmento,t_viaje_a,demandacsv=demand_file,
            vst_veh=vst_veh, vst_esp=vst_esp, vst_trb=vst_trb, costo_dist=costo_dist, costo_tpo=costo_tpo,
            cap = cap):
    demanda = []
    demanda_par = {}
    costo_total = 0
    with open(demandacsv,'r') as g:
        reader = csv.reader(g,delimiter=',',quotechar='"')
        for row in reader:
            #print(row)
            origen = row[0]
            destino = row[1]
            magnitud = row[2]
            lista = (origen,destino,magnitud)
            demanda.append(lista)
            demanda_par[(origen,destino)] = float(magnitud)
    #7 - crear segmentos de linea para cada arco de cada recorrido con carga
    global recorr_carga
    recorr_carga = {}
    costos = {}
    for linea in recorrido:
        recorr_carga[linea]={}
        for pos in range(len(recorrido[linea])-1):
            origen = recorrido[linea][pos]
            destino = recorrido[linea][pos+1]
            arco = (origen,destino)
            recorr_carga[linea][arco]=0
    del origen,destino
    #8 - Asignación de cada par OD
    carga, rutas = {}, {}
    for edge in A.edges():
        carga[edge] = 0
    costo_viaje = 0
    t_discr = {}
    costo_por_linea = {}
    for l in frecuencia:
      if len(l) == 3:
        costo_por_linea[l] = 0
    costo_por_par = []
    for par in demanda:
        t_discr[par] = [0,0]
        costos[par] = 0
        o, d = par[0], par[1]
        carga_par = float(par[2])
        #print(o,d)
        try:
            path = nx.shortest_path(L, source = o, target = d) #cada arco vale 1, así encuentro la ruta con menos trasbordos
            rutas[(o,d)] = path
        except:
            print('No se encontro una ruta entre ', o, 'y', d)
            pass
        costo_p = 0 
        costo_p = costo_p + (max(0,(len(path)-2)*vst_trb)*carga_par) #el num de trasbordos es la longitud de la ruta mínima menos 2
        for pos in range(len(path)-1):
            arco = (path[pos], path[pos+1])
            ##asignar carga a cada segmento de cada linea
            cvia, cesp = 0,0
            for linea in lineas_comunes[arco]:
                if len(linea) == 3:
                  frec_asign = float(frecuencia[linea])/frec_l_com[arco]
                  carga_asign = frec_asign * float(carga_par)
                  comienzo = arco[0]
                  pos_origen = recorrido[linea].index(comienzo)
                  final = arco[1]
                  pos_final = recorrido[linea].index(final)
                  for i in range(pos_origen, pos_final):
                      pos_comienzo = recorrido[linea].index(comienzo)
                      destino = recorrido[linea][pos_comienzo+1]
                      tupla = (comienzo,destino)
                      recorr_carga[linea][tupla]=recorr_carga[linea][tupla]+carga_asign
                      comienzo=destino
                  cvia = cvia + (segmento[arco][linea][0]-p)*carga_asign*vst_veh
                  cesp = cesp + (1/float(segmento[arco][linea][1]))*carga_asign*vst_esp
                  costo_por_linea[linea] = costo_por_linea[linea] + cvia
                  this_par = [par, arco, linea,segmento[arco][linea][0],carga_asign,vst_veh,
                              1/float(segmento[arco][linea][1]),carga_asign,vst_esp]
                  costo_por_par.append(this_par)
            costo_p = costo_p + cvia + cesp
        costo_viaje = costo_viaje + costo_p
    df_costos_por_par = pd.DataFrame(costo_por_par,columns=["par","arco","linea","tt","carga asign.","vstveh","wt",
                                                            "carga asign.","vstesp"])
    df_costos_por_par.to_excel("costos por par.xlsx")
    costo_total = costo_viaje
    tciclo = {}
    costo_op = 0
    costo_servicio = {}
    tciclo = cycle_time(B, recorrido)
    '''
    for servicio in recorrido:
        tveh = 0
        for i in range(len(recorrido[servicio])-1):
            o,d = recorrido[servicio][i],recorrido[servicio][i+1]
            #acá calculamos el tiempo de viaje en vehiculo usando las distancais del grafo B.
            tveh = tveh + nx.shortest_path_length(B, source = o, target = d,weight = 'weight')
        d = 1/60
        #y le sumamos el tiempo de detención
        tdet = d*len(recorrido[servicio])
        tciclo[servicio] = tveh + tdet
    '''
    for servicio in recorrido:
        #costo_op_serv = float(frecuencia[servicio])*tciclo[servicio]*(costo_dist*vel*tciclo[servicio]+costo_tpo*tciclo[servicio])
        costo_op_serv = float(frecuencia[servicio])*tciclo[servicio]*(costo_dist*vel+costo_tpo)
        costo_servicio[servicio] = costo_op_serv
        if len(servicio) == 3: #obsoleto si eliminamos los servicios ficticios de transbordo?
            costo_op = costo_op + costo_op_serv
    #print(costo_por_linea)
    '''
    for i in frecuencia:
      if len(i) == 3:
        print(i, recorrido[i])
        print(i, frecuencia[i])
    '''
    costo_total = costo_viaje + costo_op
    #sección de penalización 
    exc_st = {} #exceso por servicio y tramo
    for i in recorr_carga:
        if len(i) < 5:
            exc_st[i] = 0
            for tramo in recorr_carga[i]:
                exceso = max(0, recorr_carga[i][tramo] - (float(frecuencia[i])*cap))
                if exceso > exc_st[i]:
                    exc_st[i] = exceso
    exceso = 0
    for i in exc_st:
        if len(i) < 5: #filtra los servicios reales (que tienen una nomenclatura de 3 letras) y los ficticios (5)
            exceso = exceso + exc_st[i]
    costo_total = costo_total + exceso*pen_ex
    costo_discr = [costo_viaje, costo_op, exceso*pen_ex]

    return vst_esp, costo_dist, costo_tpo, vel, cap, demanda_par, t_discr, recorr_carga, rutas, costo_total, costo_servicio, exceso, exc_st, costo_discr

In [27]:
def cycle_time(B, recorrido, det=det):
  tciclo = {}
  for servicio in recorrido:
    tveh = 0
    for i in range(len(recorrido[servicio])-1):
        o,d = recorrido[servicio][i],recorrido[servicio][i+1]
        #acá calculamos el tiempo de viaje en vehiculo usando las distancais del grafo B.
        tveh = tveh + nx.shortest_path_length(B, source = o, target = d,weight = 'weight')
        #tveh = tveh + t_viaje_a[(o,d)]
    #y le sumamos el tiempo de detención
    tdet = float(det*len(recorrido[servicio]))
    #print(servicio, tveh, tdet)
    tciclo[servicio] = tveh + tdet
  return tciclo


In [28]:
"""**asignar**: Esta función ejecuta las funciones previas: leer, crear, chriquiB, costos"""

def asignar(pen_ex):
    import pandas as pd
    import networkx as nx
    import csv
    import operator
    import matplotlib.pyplot as plt
    t_viaje_a, estaciones, B = leer(dist=distance_file)
    k, p, d, A, L, frecuencia, capacidad, recorrido, segmento, lineas_comunes, ttv, tiempo_de_viaje = crear(B, dist = distance_file, lineas = line_file, demanda = demand_file)
    frec_l_com, ttv, lineas_comunes = chriquiB (frecuencia, tiempo_de_viaje, L, ttv, segmento, lineas_comunes, k, p, d)
    vst_esp, costo_dist, costo_tpo, vel, cap, demanda_par, t_discr, recorr_carga, rutas, costo_total, costo_servicio, exceso, exc_st, costo_discr = Costos (A,B,L,pen_ex,k,p,d,ttv,frec_l_com,frecuencia,lineas_comunes,recorrido,segmento,t_viaje_a,demandacsv=demand_file)
    return costo_total, exceso, rutas, exc_st, demanda_par, vst_esp, costo_dist, costo_tpo, vel, cap, lineas_comunes, recorr_carga, costo_discr


In [29]:
def get_loads(B,recorr_carga,frecuencia,recorrido,all_stop_ida, all_stop_reg):
  df_loads = pd.DataFrame(columns=['stop list']+list(recorrido.keys()))
  stop_list = recorrido[all_stop_ida] + recorrido[all_stop_reg]
  df_loads['stop list'] = stop_list
  df_loads.set_index('stop list',inplace=True)
  for i in recorr_carga[all_stop_ida]:
    df_loads.loc[i[0],all_stop_ida] = recorr_carga[all_stop_ida][i]/float(frecuencia[all_stop_ida])
  for i in recorr_carga[all_stop_reg]:
    df_loads.loc[i[0],all_stop_reg] = recorr_carga[all_stop_reg][i]/float(frecuencia[all_stop_reg])
  services = list(recorrido.keys())
  services.remove(all_stop_ida)
  services.remove(all_stop_reg)
  for s in services:
    for a in recorr_carga[s]:
      #first_pos = stop_list.index(str(a[0]))
      #last_pos = stop_list.index(str(a[1]))
      path = nx.shortest_path(B,source = a[0], target=a[1])
      for i in path[:-1]:
        df_loads.loc[i,s] = recorr_carga[s][a]/float(frecuencia[s])
  df_loads.to_excel('loads.xlsx')
  return df_loads


#Selección de paradas y optimización de frecuencias

In [30]:
"""
**create_df**: Esta función crea los dataframe necesarios para la ejecución de Simulated Annealing
"""

def create_df(frecuencia, B, recorrido):
  df = pd.read_csv(line_file, header = None)
  paradas = {}
  with open (line_file,'r',encoding='utf-8-sig') as g:
      reader = csv.reader(g,delimiter=',')
      for row in reader:
          secuencia = []
          for parada in range(3,len(row)):
              parada_actual = (row[parada])
              secuencia.append(parada_actual)
          paradas[row[0]] = secuencia
  tveh = cycle_time(B, recorrido)
  '''
  tveh = {}
  for servicio in recorrido:
      tvehs = 0
      for i in range(len(recorrido[servicio])-1):
          o,d = recorrido[servicio][i],recorrido[servicio][i+1]
          tvehs = tvehs + nx.shortest_path_length(B, source = o, target = d,weight = 'weight')
      d = 1/60
      tdet = d*len(recorrido[servicio])
      if len(servicio)<5:
          tveh[servicio] = tvehs + tdet 
  '''
  df_freq = pd.DataFrame()
  df_freq['Nombre'] = df[0]
  #df_freq.set_index('Nombre')
  df_freq['Frecuencia'] = df[1]
  df_freq['Tiempo de ciclo']=list(tveh.values())
  df_freq['Flota asignada']=df_freq['Frecuencia']*df_freq['Tiempo de ciclo']
  df_freq['Flota mínima'] = (fmin*df_freq['Tiempo de ciclo']).apply(np.ceil)
  if punto_inicial == 'MIN':
    df_freq['Flota asignada'] = df_freq['Flota mínima']
    df_freq['Frecuencia'] = df_freq['Flota asignada'] / df_freq['Tiempo de ciclo']
    df, df_freq, frecuencia = Escribir (df, df_freq, frecuencia, B, recorrido)
  if punto_inicial == 'NOC':
    df, df_freq, frecuencia = Escribir (df, df_freq, frecuencia, B, recorrido)
  if punto_inicial == 'REP':
    nspb = flota / len(df_freq['Nombre'])
    df_freq['Flota asignada'] = nspb
    df_freq['Frecuencia'] = df_freq['Flota asignada'] / df_freq['Tiempo de ciclo']
    df, df_freq, frecuencia = Escribir (df, df_freq, frecuencia,B, recorrido)
  if punto_inicial == 'SIN':
    nspb = flota / 2
    flotas =  list(df_freq['Flota mínima'])
    flotas[0], flotas[1] = nspb, nspb
    df_freq['Flota asignada'] = flotas
    df, df_freq, frecuencia = Escribir (df, df_freq, frecuencia,B, recorrido)
  if punto_inicial == 'FRR':
    df_freq['Frecuencia'] = 10
    df_freq['Flota asignada'] = df_freq['Frecuencia'] * df_freq['Tiempo de ciclo']

    df, df_freq, frecuencia = Escribir (df, df_freq, frecuencia,B, recorrido)
  #print(df_freq)
  return df, df_freq, frecuencia

In [31]:
"""**Escribir** es una función interna que se utiliza para actualizar la información en cada iteración."""

def Escribir (df, df_freq, frecuencia, B, recorrido):
    tciclo = cycle_time(B, recorrido)
    df_freq['Tiempo de ciclo']=list(tciclo.values())
    df_freq['Frecuencia'].replace({0:0.01}, inplace = True)
    df_freq['Flota asignada'] = df_freq['Tiempo de ciclo']*df_freq['Frecuencia']
    df[1] = df_freq['Frecuencia']
    df.to_csv(line_file, index = False, header = False, na_rep='')
    rowlist = []
    with open (line_file,'r') as g:
        reader = csv.reader(g,delimiter=',')
        for row in reader:
            while '' in row:
                row.remove('')
            rowlist.append(row)
    with open (line_file,'w') as h:
        w = csv.writer(h,delimiter=',')
        w.writerows(rowlist)
    for i in df_freq.iterrows():
      frecuencia[i[1][0]] = i[1][1]
    #print(frecuencia)
    return df, df_freq, frecuencia

In [33]:
"""**Ciclos** crea un grafo R que contiene un arco por cada servicio. 
Encuentra todos los ciclos de servicios, y para cada uno de ellos encuentra la 
frecuencia a alterar en cada uno de los serivcios que lo componen para mantener 
el equilibrio en flujo de buses en el sistema.

Esta función, junto con Poner y Quitar, fueron reemplazadas por un PFMC.

La función impacto_paradas aún se utiliza.

"""

def Ciclos (recorrido, estaciones, df_freq, df_poner, df_quitar):
    #df_freq = df_freq.sort_index()
    #df_poner = df_poner.sort_index()
    #df_quitar = df_quitar.sort_index()
    global dict_ciclos
    global costo_ciclos
    R = nx.MultiDiGraph()
    for s in recorrido: 
        if len(s) < 5:
            index = df_freq.loc[df_freq['Nombre'] == s].index[0]
            inicio, fin = recorrido[s][0],recorrido[s][-1]
            R.add_edge(inicio, fin,recorrido = s)
    #añadir arcos de trasbordo
    '''
    for e in estaciones:
        if e[-1] == 'i':
            R.add_edge(e, e[:-1]+'r', recorrido = 'x')
        if e[-1] == 'r':
            R.add_edge(e, e[:-1]+'i', recorrido = 'x')
    '''
    #encontremos ciclos
    cycles = list(nx.simple_cycles(R))
    #print(cycles)

    for i in cycles[:]: 
        if len(i) == 2: 
            if i[0][:-1] == i[1][:-1]:
                cycles.remove(i)
    for i in cycles:
        i.append(i[0])
    nombre_ciclo = 0
    #ahora veamos todas las posibles combinaciones para cada ciclo (un ciclo puede tener más de un servicio)
    dict_ciclos_gen = {}
    #dict con info de los ciclos: 0 - ruta, 1 - servicio, 
    for ciclo_generico in cycles:
        combinaciones = []
        dict_ciclos_gen[nombre_ciclo] = [ciclo_generico]
        for arco in range(len(ciclo_generico)-1):
            inicio = ciclo_generico[arco]
            fin = ciclo_generico[arco+1]
            dict_data = R.get_edge_data(inicio,fin)
            opciones = []
            for servicio in dict_data:
                opciones.append(dict_data[servicio]['recorrido'])
            combinaciones.append(opciones)
        dict_ciclos_gen[nombre_ciclo].append(combinaciones)
        nombre_ciclo = nombre_ciclo + 1
    for nombre_ciclo in dict_ciclos_gen:
        dict_ciclos_gen[nombre_ciclo].append(list(itertools.product(*dict_ciclos_gen[nombre_ciclo][1])))
    #ahora creemos un id para cada ciclo único 
    dict_ciclos = {}
    cod_ciclo = 0
    for ciclo_gen in dict_ciclos_gen:
        for ciclo in dict_ciclos_gen[ciclo_gen][2]:
            dict_ciclos[cod_ciclo] = list(ciclo)
            dict_ciclos[cod_ciclo] = list(ciclo)
            cod_ciclo = cod_ciclo + 1
    #descarto los arcos de transbordo
    for ciclo in dict_ciclos:
        for etapa in dict_ciclos[ciclo][:]:
            if etapa == 'x':
                dict_ciclos[ciclo].remove(etapa)
    #ahora veamos el costo de cada ciclo
    costo_ciclos = {}
    for ciclo in dict_ciclos: 
        costo_poner, costo_quitar = 0, 0
        #añado o quito dos buses al servicio inicial y causo un desbalance
        serv_inicial = dict_ciclos[ciclo][0]
        index = df_poner.loc[df_freq['Nombre'] == serv_inicial].index[0]
        desbal_f = 2 / float(df_freq.iloc[index,2])
        num_buses = [2]
        costo_poner = df_poner.iloc[index,12]
        costo_quitar = df_quitar.iloc[index,14]
        for servicio in dict_ciclos[ciclo][1:]:
            index_p = df_poner.loc[df_freq['Nombre'] == servicio].index[0]
            index_q = df_quitar.loc[df_freq['Nombre'] == servicio].index[0]
            #calculo el numero de buses que necesito para compensar el desbalance
            buses = desbal_f * df_freq.iloc[index_p,2]
            #calculo cuántos buses requiere este ciclo
            num_buses.append(buses)
            porcion_delta = buses/2
            #print(porcion_delta)
            delta_costo_poner = df_poner.iloc[index_p,12]*porcion_delta
            delta_costo_quitar = df_quitar.iloc[index_q,14]*porcion_delta
            costo_poner = costo_poner + delta_costo_poner
            costo_quitar = costo_quitar + delta_costo_quitar
        costo_ciclos[ciclo] = {}
        #guardo el costo de poner o quitar dos ciclos, y cuántos buses requiere
        costo_ciclos[ciclo]['Poner'] = [costo_poner, num_buses]
        costo_ciclos[ciclo]['Quitar'] = [costo_quitar, num_buses]
    return R,dict_ciclos, costo_ciclos


"""**impacto_paradas** estima el impacto en la función objetivo de poner o quitar una parada de un servicio. """

def impacto_paradas (vst_veh, vst_esp, costo_tpo, recorrido, recorr_carga, demanda_par, rutas, B, last_choices, all_stop_ida = all_stop_ida, all_stop_reg = all_stop_reg): 
    estaciones_orden = []
    for r in recorrido:
      for px in recorrido[r]:
        if px not in estaciones_orden:
          estaciones_orden.append(px)
    #estaciones_orden = list(set(estaciones_orden))
    #print(estaciones_orden)
    #solo se usarán los servicios con len(id_serv) == 3
    servicios = list(filter(lambda x: len(x) == 3, list(recorrido.keys())))
    cromosoma = pd.DataFrame(columns = estaciones_orden)
    #print(estaciones_orden)
    for s in servicios:
        sentido = recorrido[s][0][-1]
        first, last = estaciones_orden.index(recorrido[s][0]), estaciones_orden.index(recorrido[s][-1])
        chromo = []
        #1 = estación visitada 
        #0 = estación no visitada 
        #X = estación inhabilitada
        for est in list(cromosoma.columns):
            if est in recorrido[s]:
              if recorrido[s].index(est) != 0 and recorrido[s].index(est) != len(recorrido[s])-1:
                chromo.append('1')
              elif recorrido[s].index(est) == 0 or recorrido[s].index(est) == len(recorrido[s])-1:
                chromo.append('T') #estacion terminal
            elif estaciones_orden.index(est) < first or estaciones_orden.index(est) > last:
                chromo.append('X')
            else:
                chromo.append('0')
        cromosoma.loc[s] = chromo
    matriz_impacto = cromosoma.copy(deep=True)
    matriz_impacto.drop([all_stop_ida,all_stop_reg], inplace=True)
    #dataframe con la carga esperada de cada servicio en cada estacion
    #la carga reportada en la celda (servicio, estacion) indica la carga esperada al salir de  estación 
    carga = {}
    for serv in servicios:
        carga_serv = [0]*len(cromosoma.columns)
        for tramo in recorr_carga[serv]:
            ini, fin = estaciones_orden.index(tramo[0]), estaciones_orden.index(tramo[1])
            for est in range(ini,fin):
                carga_serv[est] = recorr_carga[serv][tramo]
        carga[serv] = carga_serv
    carga = pd.DataFrame.from_dict(carga, orient = 'index', columns = cromosoma.columns)
    #dataframe con la demanda total de cada servicio
    demanda_servicio = {}
    for servicio in servicios:
        demanda_servicio[servicio] = 0
    #secciones es muy parecido a rutas, solo que muestra las secciones de ruta en forma de tupla
    #acá estoy hablando de lineas comunes solamente
    
    secciones = {}
    for par in demanda_par:
        if par[0][:-1]!=par[1][:-1]:
            secciones[par]=[]
            for pos in range(len(rutas[par])-1):
                secciones[par].append((rutas[par][pos],rutas[par][pos+1]))
            for seccion in secciones[par]:
                for servicio in lineas_comunes[seccion]:
                        demanda_servicio[servicio] += demanda_par[par]
    
    #servicios disponibles para cada par od  (todos los disponibles)
    serv_disp = {}
    for par in demanda_par:
        serv_disp[par] = []
        for servicio in recorrido: 
            if par[0] and par[1] in recorrido[servicio]:
                serv_disp[par].append(servicio)
    ##ESTIMAMOS EL IMPACTO DE QUITAR UNA PARADA
    for servicio in servicios:
        if servicio not in [all_stop_ida, all_stop_reg]:
            for estacion in list(cromosoma.columns):
                if cromosoma.loc[servicio,estacion] == '1':
                    #impacto guarda los impactos estimados en: tiempo de detención, tiempo de espera, transbordos, penalidad x exceso, costo de operación
                    impacto = [0,0,0,0,0]
                    #leemos la orden
                    operacion = 'Q'
                    #impacto de la operación en el tiempo de viaje en vehículo
                    indice_estacion = estaciones_orden.index(estacion)-1
                    #el impacto está dado por el tiempo ahorrado por no detenerse una vez transformado a dinero
                    impacto[0] = carga.loc[servicio,estaciones_orden[indice_estacion]]*d*-vst_veh
                    #impacto de la operación en el tiempo de espera
                    index = df_freq.loc[df_freq['Nombre'] == servicio].index[0]
                    #efecto positivo: baja el tiempo de ciclo y por tanto el tiempo de espera para todo el servicio
                    impacto[1] = vst_esp*(-d/df_freq.iloc[index,3])*demanda_servicio[servicio] 
                    #efecto negativo: La estación afectada ve reducida su frecuencia efectiva
                    pares_afectados = {}
                    for par in secciones:
                        if servicio in lineas_comunes[par] and estacion in par:
                            pares_afectados[par] = 0
                    for par in demanda_par:
                        if par[0][:-1]!=par[1][:-1]:
                            for seccion in secciones[par]:
                                if seccion in list(pares_afectados.keys()):
                                    pares_afectados[seccion] += demanda_par[par]
                    impacto_te = {}
                    for par in pares_afectados:
                        impacto_te[par] = 0
                        f, f0 = 0,0
                        for l in lineas_comunes[par]:
                            f0 += float(frecuencia[l])
                            if l != servicio:
                                f += float(frecuencia[l])
                        try:
                            #ojo, si el conjunto de lineas comunes queda vacio, omito este par. 
                            #lo hago porque la frec. del conj. de LA  = 0.
                            impacto_te[par] += (1/f)-(1/f0)
                        except:
                            pass
                    impacto_tef = 0
                    for par in pares_afectados:
                        impacto_tef += pares_afectados[par]*impacto_te[par]
                    #entonces el impacto negativo es el tiempo de espera adicional en/hacia la estación afectada.
                    impacto[1] += impacto_tef*vst_esp
                    
                    #impacto en el número de transbordos
                    #seleccionamos las secciones de ruta que quedarán sin una conexión directa y requerirán un trasbordo adicional
                    huerfanos = []
                    for par in serv_disp:
                        if len(serv_disp[par]) == 1 and servicio in serv_disp[par] and estacion in par:
                            huerfanos.append(par)
                    #ahora vemos qué pares od usan estas secciones de ruta 
                    pares_huerfanos = []
                    for par in secciones:
                        for seccion in huerfanos:
                            if seccion in secciones[par]:
                                pares_huerfanos.append(par)
                    demanda_huerfana = 0
                    for par in pares_huerfanos:
                        demanda_huerfana += demanda_par[par]
                    impacto[2] = p * demanda_huerfana
                    #impacto en el costo de operación
                    impacto[4] = d * -costo_tpo * float(frecuencia[servicio])
                    #print(impacto)
                    impacto_movimiento = sum(impacto)
                    matriz_impacto.loc[servicio,estacion] = impacto_movimiento
                    #print('Q',servicio,estacion,impacto_movimiento)
                else:
                    pass

        ##ESTIMAMOS EL IMPACTO DE AÑADIR UNA PARADA
    for servicio in servicios:
        if servicio not in [all_stop_ida, all_stop_reg]:
            for estacion in list(cromosoma.columns):
                if cromosoma.loc[servicio,estacion] == '0':
                    #impacto guarda los impactos estimados en: tiempo de detención, tiempo de espera, transbordos, penalidad x exceso, costo de operación
                    impacto = [0,0,0,0,0]
                    #leemos la orden
                    operacion = 'P'
                    #impacto de la operación en el tiempo de viaje en vehículo
                    indice_estacion = estaciones_orden.index(estacion)-1
                    #el impacto está dado por el tiempo ahorrado por no detenerse una vez transformado a dinero
                    impacto[0] = carga.loc[servicio,estaciones_orden[indice_estacion]]*d*vst_veh
                    #impacto de la operación en el tiempo de espera
                    index = df_freq.loc[df_freq['Nombre'] == servicio].index[0]
                    #efecto negativo: aumenta el tiempo de ciclo y por tanto el tiempo de espera para todo el servicio
                    impacto[1] = vst_esp*(d/df_freq.iloc[index,3])*demanda_servicio[servicio] 
                    #efecto positivo: La estación afectada aumenta su frecuencia efectiva
                    pares_afectados, impacto_te = {}, {}
                    for par in secciones:
                        if estacion in par[0] or estacion in par[1]:
                            #vamos a calcular el tiempo esperado de viaje antes de añadir esta parada 
                            #si el tiempo de viaje disminuye, entonces el movimiento añadirá el servicio estudiado al conj. de lineas atractivas.
                            num, den = 1, 0
                            for l in lineas_comunes[par]:
                                num += segmento[par][l][0]*float(frecuencia[l])
                                den += float(frecuencia[l])
                                tev = num/den
                            tvh = nx.shortest_path_length(B, source = par[0], target = par[1], weight = 'weight')
                            num_paradas = estaciones_orden.index(par[1]) - estaciones_orden.index(par[0])
                            for i in range(estaciones_orden.index(par[0]),estaciones_orden.index(par[0])+num_paradas):
                                if estaciones_orden[i] not in recorrido[servicio]:
                                    num_paradas -= 1
                            tvh += num_paradas*d
                            numh = num + tvh*float(frecuencia[servicio])
                            denh = den + float(frecuencia[servicio])
                            tevh = numh/denh
                            if tevh < tev:
                                pares_afectados[par] = demanda_par[par]
                                impacto_te[par] = tevh-tev
                    for par in demanda_par:
                        if par[0][:-1]!=par[1][:-1]:
                            for seccion in secciones[par]:
                                if seccion in list(pares_afectados.keys()):
                                    pares_afectados[seccion] += demanda_par[par]
                    impacto_tef = 0
                    for par in pares_afectados:
                        impacto_tef += pares_afectados[par]*impacto_te[par]
                    #entonces el impacto positivo es la reducción en el tiemop de viaje esperado en/hacia la estación afectada.
                    impacto[1] += impacto_tef*vst_esp
                    #impacto en el número de transbordos: nulo
                    impacto[2] = 0
                    #impacto en el costo de operación
                    impacto[4] = d * costo_tpo * float(frecuencia[servicio])
                    impacto_movimiento = sum(impacto)
                    matriz_impacto.loc[servicio,estacion] = impacto_movimiento
                    #print('P',servicio,estacion,impacto_movimiento)
                else:
                    pass
    #print(matriz_impacto)          
    for choice in last_choices:
      servicio = choice[0]
      estacion = choice[1]
      matriz_impacto.loc[servicio,estacion] = 'X'

    impactos = matriz_impacto.values.tolist()
    impactos_norm = []
    for s in impactos: 
        impactos_norm.append([0 if x == 'X' else 0 if x == 'T' else x for x in s])

        
    candidatos = pd.DataFrame(columns = ['Estación','Impacto'], index = matriz_impacto.index)
    mejor = []
    for s in impactos_norm: 
        mejor.append(min(s))
    candidatos['Impacto'] = mejor
    estaciones_cand = []
    for i in range(len(mejor)): 
         index = impactos_norm[i].index(mejor[i])
         estaciones_cand.append(estaciones_orden[index])
    candidatos['Estación'] = estaciones_cand
    #print(candidatos)
    gen = []
    for i in cromosoma.iterrows():
      gen.append(i[0])
      row = list(i[1])
      out0 = [x for x in row]
      out = ['0' if x == 'X' else x for x in out0]
      gen = gen + out
    return matriz_impacto, candidatos, estaciones_orden, gen

In [37]:
last_choices = []
matriz_impacto, candidatos, estaciones_orden, gen = impacto_paradas(vst_veh, vst_esp, costo_tpo, recorrido, recorr_carga, demanda_par, rutas, B, last_choices, all_stop_ida = all_stop_ida, all_stop_reg = all_stop_reg)

In [38]:
matriz_impacto

,55334,60505,55335,55047,55059,55079,50733,50815,50880,50932,51001,51079,51143,51224,51277,51360,51394,51471,51473,51519,51557,51605,51632,54364,51745,51839,51945,52047,52109,52159,52240,52349,52432,52522,52628,53826,61046,52865,52952,53053,53036,53011,52922,52815,53825,52651,52520,52431,52347,52239,52158,52107,52046,51943,51838,51743,60617,51630,51606,51517,51469,51392,51359,53968,51276,54180,51142,51080,51000,50931,50879,50814,50732,55078,55057,55046,55333
E1S,T,1339.537332,238.189878,985.87812,1879.118006,-60.862364,1071.402394,-60.887466,1716.753591,477.973642,2742.63164,724.562493,1718.210041,-103.312283,291.113865,-106.552644,1454.014541,-100.882832,465.714516,-105.537644,600.948803,7112.592318,-77.484353,-77.484353,160.31383,1095.736754,1731.724244,-89.109923,1550.900453,1269.185177,352.909682,1591.694118,-95.084235,550.823456,240.497734,394.458021,-99.102762,468.557371,1361.310777,-82.479681,T,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X
E1N,T,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,X,T,-42.769412,222.926882,954.597251,255.596772,2999.32782,935.838625,697.33966,2427.80511,1490.180312,71.525341,1474.289611,1416.338469,1857.041243,1326.657545,227.823854,97.171022,660.371203,9132.962628,159.003518,1288.616941,1117.212873,-68.285203,-68.285203,-68.285203,337.300191,445.18372,218.883152,-63.05959,1209.227115,2944.845471,-27.982063,-27.982063,-27.982063,459.703485,-31.988584,584.825363


#Simulated Annealing

In [35]:
#Función auxiliar
#Esta función encuentra el valor de la derivada de los costos sociales definida 
#para un vector direccional (instruccion). X es el tamaño del paso

#el desarrollo de la derivada está en el archivo Paso óptimo PFMC.docx

#corremos las primeras celdas para poder tener un vector inicial y poder definir
#las funciones con df_freq predeterminado
pen_ex = 1
t_viaje_a, estaciones, B = leer(dist=distance_file)
k, p, d, A, L, frecuencia, capacidad, recorrido, segmento, lineas_comunes, ttv, tiempo_de_viaje = crear(B, dist = distance_file, lineas = line_file, demanda = demand_file)
frec_l_com, ttv, lineas_comunes = chriquiB (frecuencia, tiempo_de_viaje, L, ttv, segmento, lineas_comunes, k, p, d)
vst_esp, costo_dist, costo_tpo, vel, cap, demanda_par, t_discr, recorr_carga, rutas, costo_total, costo_servicio, exceso, exc_st, costo_discr = Costos (A,B,L,pen_ex,k,p,d,ttv,frec_l_com,frecuencia,lineas_comunes,recorrido,segmento,t_viaje_a,demandacsv=demand_file)
df, df_freq, frecuencia = create_df (frecuencia, B, recorrido)
costo_total, exceso, rutas, exc_st, demanda_par, vst_esp, costo_dist, costo_tpo, vel, cap, lineas_comunes, recorr_carga, costo_discr = asignar (pen_ex)



#OJO: vst_esp = 3 * vst_veh. Si cambia el factor, cambiar en los dos lugares corresp

def dZ (x,instruccion,df_freq=df_freq, demanda_par=demanda_par, lineas_comunes=lineas_comunes,
        frecuencia=frecuencia, segmento=segmento, 
        costo_tpo = costo_tpo, costo_dist = costo_dist, exc_st=exc_st, 
        vel =vel, cap=cap, vst_veh=vst_esp*(1/2)):
  #derivada de los costos del operador
  dO = 0
  for l in frecuencia:
    if len(l) == 3:
      index = df_freq.index[df_freq['Nombre'] == l].tolist()[0]
      tc = df_freq.loc[index]['Tiempo de ciclo']
      c = tc*(costo_tpo*tc+costo_dist*vel*tc)
      dO += c*instruccion[l]
  rows, index = [], []
  #derivada de los costos de los usuarios
  dU = 0
  #calculamos A, B, C, D (factores para la derivada)
  for par in demanda_par:
    if par[0][:-1] != par[1][:-1]:
      demanda = demanda_par[par]
      k = 1 #parámetro de regularidad
      A, B, C, D = 3*k,0,0,0
      lc = lineas_comunes[par]
      for l in lc:
        A += frecuencia[l]*segmento[par][l][0]
        B += frecuencia[l]
        C += instruccion[l]
        D += instruccion[l]*segmento[par][l][0]
      rows.append([par[0],par[1],A,B,C,D]) 
  calculo_paso = pd.DataFrame(rows, columns = ['Origen','Destino','A','B','C','D'])
  calculo_paso['Num'] = (calculo_paso['B']*calculo_paso['D'])-(calculo_paso['A']*calculo_paso['C'])
  for par in demanda_par:
    if par[0][:-1] != par[1][:-1]:
      demanda = demanda_par[par]
      index = calculo_paso.index[(calculo_paso['Origen']==par[0]) & (calculo_paso['Destino']==par[1])].tolist()[0]
      num = calculo_paso.iloc[index]['Num']
      den = (calculo_paso.iloc[index]['B']+ x*calculo_paso.iloc[index]['C'])**2
      dU += demanda*(num/den)*vst_veh
  #derivada de las penalizaciones
  dP = 0
  for l in frecuencia:
    if len(l) == 3:
      if exc_st[l] > 0:
        dP -= instruccion[l]*cap
  return dO+dU+dP

def PasoOptimo (instruccion,df_freq,a=0):
  
  #primero hallamos la cota máxima del paso, dada por la frecuencia disponible
  #que tiene cada servicio 

  #paso_max = pd.DataFrame()
  #acá vemos cuánta frecuencia tiene disponible cada servicio
  #paso_max['X'] = df_freq['Frecuencia'] - (df_freq['Flota mínima']/df_freq['Tiempo de ciclo'])
  #b = max(list(paso_max['X'].values))
  b = min(df_freq['Frecuencia'])
  #b = max(b-1, 0) #frecuencia minima deseada, 1 en este caso
  
  while b-a > 0.1:
    r = (a+b)/2
    y_a, y_b, y_r = dZ(a,instruccion), dZ(b,instruccion), dZ(r,instruccion)

    if y_a*y_r >= 0:
      a = r
    
    if y_b*y_r > 0:
      b = r

    print('a {}  f(a) {} b {} f(b) {}'.format(a,y_a,b,y_b))
  return (a+b)/2

In [36]:
"""**sim_annealing_ciclos** Ejecuta la búsqueda de frecuencias y selección de paradas. Para ello ejecuta alternadamente iteraciones donde se altera la frecuencia de un ciclo de servicios, con una donde se altera una parada de autobús. Con el objetivo de escapar de óptimos locales, la rutina se ejecuta aceptando ciertos movimientos que empeoran la solución incumbente (técnica conocida como Simulated Annealing). """

def Sim_annealing_ciclos (df, df_freq, estaciones):
    print('{:12} {:12} {:13} {:12}% {:10} {:10} {:10} {:10} {:10} {:10}'
    .format('Temperatura', 'Operación', 'Costo', 'Variación', 'C. usuarios', 
            'C. operador', 'Penalización exc.', 'Flota asign.', 'Prob. acept.', 'Movimiento'))
    resultados = pd.DataFrame(columns = ['Temperatura', 'Operación', 'Costo', 'Variación', 'Decisión', 'C. usuarios', 
                      'C. operador', 'Penalización exc.', 'Flota asign.', 'Prob. acept.', 'Movimiento'])
    val_derivadas = pd.DataFrame(columns = ['Temperatura','D. Operador','D. Espera','D. Penalidad','D. Viaje'])
    t0 = t_input
    t = t0
    global temp 
    global costos
    global recorrido
    temp = [] #spam?
    costos = [] #spam?
    global excesos 
    excesos = [] #spam?
    choices = [] #spam?
    global variacion
    variacion = [] #spam?
    global flota_asign
    global recorr_carga
    flota_asign = [] #spam?
    rows = [] #spam?
    costo_componentes = [] #spam?
    historial_costos = [] #spam?
    probs = [] #spam?
    probs2 = [] #spam?
    pen_ex = math.exp(-(math.log(10000)/t0)*(t-t0))
    historial_frecuencias = pd.DataFrame()
    #randoms, exps = [], []
    flota_disp = 4000
    costo, exceso, rutas, exc_st, demanda_par, vst_esp, costo_dist, costo_tpo, vel, cap, lineas_comunes, recorr_carga, costo_discr = asignar(pen_ex)
    it = 1
    output = {} #spam?
    t_viaje_a, estaciones, B = leer(dist=distance_file)
    k, p, d, A, L, frecuencia, capacidad, recorrido, segmento, lineas_comunes, ttv, tiempo_de_viaje = crear(B, dist = distance_file, lineas = line_file, demanda = demand_file)
    df, df_freq, frecuencia = create_df (frecuencia, B, recorrido)
    x = -1000 #parámetro que regula la probabilidad de aceptar un movimiento que empeore 
    last_choices = []
    desvest = 11
    while t > 10000 and desvest > 10:
      #print(t)
      #ITERACIÓN DE AÑADIR / QUITAR BUSES
      if (it % 2) != 0:
          #print(df_freq)
          print('FREQUENCY MOVEMENT - It. {}. - Temp. {}').format(it,t)
          '''
          for i in frecuencia:
            if len(i) == 3:
              print(i, frecuencia[i])
          '''
          pen_ex = math.exp(-(math.log(10000)/t0)*(t-t0))
          #asignamos
          #pen_ex = 10000
          #print(pen_ex)
          #print(round(pen_ex))
          df_p = df.copy(deep=True)
          df_freq_p = df_freq.copy(deep=True)
          # exploramos la función de vecindario
          derivada = {}
          for l in frecuencia:
            if len(l) == 3: 
              #derivadas de la función objetivo para conocer su cambio frente a cambios en f
              #derivada calculada en página 93 de mi cuaderno
              #derivada de [costos_operador,tiempo_espera, penalidad, tiempo_viaje], suma
              #derivada tiempo_viaje despreciable
              derivada[l] = [[0,0,0,0], 0]
              #averiguo el index de cada servicio
              index = df_freq.index[df_freq['Nombre'] == l].tolist()[0]
              #print(df_freq)
              tc = df_freq.loc[index]['Tiempo de ciclo']
              #derivada del costo del operador
              derivada[l][0][0] = tc*(costo_tpo*tc+costo_dist*vel*tc)
              #derivada del tiempo de espera 
              ies, ivi = 0, 0
              for par in demanda_par:
                #experimentar con secciones de ruta por cada par
                if par[0][:-1] != par[1][:-1] and l in lineas_comunes[par]:
                  F = 0
                  tf = 0
                  try:
                    lc = lineas_comunes[par]
                    for lca in lc:
                      F = F + frecuencia[lca]
                      tf = tf + segmento[par][lca][0]*frecuencia[lca]
                  except:
                    pass
                  ies = ies + ((demanda_par[par]*(frecuencia[l]/F)*vst_esp)*((-k)/(F**2)))
                  num1 = float(segmento[par][l][0])*F
                  ivi = ivi + (demanda_par[par]*(frecuencia[l]/F)*vst_veh*((num1-tf)/F**2))
              derivada[l][0][1] = derivada[l][0][1] + ies #avancé esto un tab
              derivada[l][0][3] = derivada[l][0][3] + ivi
              if exc_st[l] > 0:
                derivada[l][0][2] = -cap*pen_ex
              derivada[l][1] = sum(derivada[l][0])  
                
          #print(derivada)
          derivadaporcosto = [t,0,0,0,0]
          #guardamos el valor de la derivada por cuatro componentes:
          #d. operador, d. t . espera, penalidad, t. viaje
          C = nx.MultiDiGraph()
          for l in derivada:
            index = df_freq.loc[df_freq['Nombre'] == l].index[0]
            if df_freq.iloc[index]['Frecuencia'] <= 1:
              #Los servicios que tienen frecuencia menor a 1, solo pueden aumentar
              #su frecuencia.
              C.add_edge(recorrido[l][0], recorrido[l][-1], capacity = 1, weight = derivada[l][1], key = l+'+')
            else:
              C.add_edge(recorrido[l][0], recorrido[l][-1], capacity = 1, weight = derivada[l][1], key = l+'+')
              C.add_edge(recorrido[l][-1], recorrido[l][0], capacity = 1, weight = -derivada[l][1], key = l+'-')
          delta, solucion = nx.algorithms.flow.network_simplex(C, capacity = 'capacity', weight = 'weight')
          #print(solucion)
          #print('delta {} solucion {}'.format(delta,solucion))
          #delta es el immpacto estimado con la dirección de solución
          #siempre va a ser negativo, excepto cuando todos los costos de los arcos sean cero
          #instruccion es el vector direccional de cambio
          instruccion = {}
          for servicio in recorrido:
            instruccion[servicio] = 0
            for origen in solucion:
              for destino in solucion[origen]:
                for bit in solucion[origen][destino]:
                  if bit[:-1] == servicio:
                    if solucion[origen][destino][bit] == 1:
                      if bit[-1] == '+':
                        instruccion[servicio] = 1
                      if bit[-1] == '-':
                        instruccion[servicio] = -1

          #print(instruccion)
          #determinamos el paso óptimo
          #el paso máximo es b=50.
          paso = PasoOptimo(instruccion, df_freq)
          #print(paso)
          #print(instruccion)
          direction = {}
          for ins in instruccion: 
            servicio = ins
            if instruccion[ins] == 1:
              direction[servicio] = '+'
            elif instruccion[ins] == -1:
              direction[servicio] = '-'
          for servicio in direction:
            if direction[servicio] == '+':
              index = df_freq.loc[df_freq['Nombre'] == servicio].index[0]
              #acá está la magia: se cambia la frecuencia de la línea
              nueva = float(df_freq.iloc[index]['Frecuencia']) + paso
              #print(servicio,float(df_freq.iloc[index]['Frecuencia']),nueva,paso)
              df_freq.Frecuencia = df_freq.Frecuencia.astype(float)
              df_freq.at[index,'Frecuencia'] = nueva
              #print(df_freq.loc[index,'Frecuencia'])
              #este bloque solo es para guardar las derivadas
              derivadaporcosto[1] += derivada[servicio][0][0]
              derivadaporcosto[2] += derivada[servicio][0][1]
              derivadaporcosto[3] += derivada[servicio][0][2]
              derivadaporcosto[4] += derivada[servicio][0][3]
            elif direction[servicio] == '-':
              index = df_freq.loc[df_freq['Nombre'] == servicio].index[0]
              nueva = float(df_freq.iloc[index]['Frecuencia']) - paso
              df_freq.Frecuencia = df_freq.Frecuencia.astype(float)
              df_freq.at[index,'Frecuencia'] = float(nueva)
              derivadaporcosto[1] -= derivada[servicio][0][0]
              derivadaporcosto[2] -= derivada[servicio][0][1]
              derivadaporcosto[3] -= derivada[servicio][0][2]
              derivadaporcosto[4] -= derivada[servicio][0][3]
            else:
              print('No se pudo reconocer la siguiente orden: {}'.format(direction))
          #print(df_freq)
          val_derivadas = val_derivadas.append(pd.DataFrame([derivadaporcosto], columns=val_derivadas.columns))
          df_freq['Flota asignada'] = df_freq['Frecuencia']*df_freq['Tiempo de ciclo']
          df, df_freq, frecuencia = Escribir (df, df_freq, frecuencia,B, recorrido)
          choices.append(str(instruccion))
          probs.append('N.A.')
          operador = 'Frecuencia'
          #print('Probando nueva solución')
          costo_nuevo, exceso_new, rutas, exc_st, demanda_par, vst_esp, costo_dist, costo_tpo, vel, cap, lineas_comunes, recorr_carga, costo_discr_nvo = asignar(pen_ex)
          cost_diff = costo_nuevo - costo
          perc_diff = (cost_diff/costo)*100
          variacion.append(perc_diff)
          #print('El costo varió un {}%'.format(perc_diff))
          if costo_nuevo > costo:
              kpd = x * perc_diff #tolerancia al empeoramiento
              #entre mas grande el numero, mas despacio baja la probabilidad de ser aceptado
              #no olvidar cambiar el kpd en las iteraciones de seleccion de parada
              randomn = rand.uniform(0,1)
              if randomn > math.exp(kpd/t):
                  df = df_p.copy(deep=True)
                  df_freq = df_freq_p.copy(deep=True)
                  df_freq['Frecuencia'] = df_freq_p['Frecuencia']
                  df, df_freq, frecuencia = Escribir(df, df_freq, frecuencia,B, recorrido)
                  #print ('El costo subió y no se aceptó el cambio')
                  decision = 'Rechazar'
              if randomn <= math.exp(kpd/t):
                  df, df_freq, frecuencia = Escribir(df, df_freq, frecuencia,B, recorrido)
                  #print ('El costo subió y se permitió el cambio')
                  costo = costo_nuevo
                  exceso = exceso_new
                  costo_discr = costo_discr_nvo
                  decision = 'Aprobar'
          else: 
              #print('El costo disminuyó')
              df, df_freq, frecuencia = Escribir (df, df_freq, frecuencia,B, recorrido)
              costo = costo_nuevo
              exceso = exceso_new
              costo_discr = costo_discr_nvo
              decision = 'Descendió'
          '''
          last = recorrido[choice][-1]
          if last[-1] == 'i':
              first = last[-1]+'r'
          if last[-1] == 'r':
              first = last[-1]+'i'
          '''
          print(df_freq)
          historial_costos.append(costo)
          #print(df_freq)

      #ITERACION DE AGREGAR/QUITAR PARADAS
      if (it % 2) == 0:
        print('STOP MOVEMENT - It. {}. - Temp. {}').format(it,t)

        #print('Iteraciónn paradas')
        #pen_ex = math.exp(-(math.log(10000)/t0)*(t-t0))
        #print('Asignación previa')
        #we update last choices so they aren't available now
        if len(last_choices) >= 3:
          last_choices = last_choices[-3:]
        #print(last_choices)
        costo, exceso, rutas, exc_st, demanda_par, vst_esp, costo_dist, costo_tpo, vel, cap, lineas_comunes, recorr_carga, costo_discr = asignar (pen_ex)        
        recorridoh = recorrido
        vst_veh = (1/2) * vst_esp
        matriz_impacto, candidatos, estaciones_orden, gen = impacto_paradas (vst_veh, vst_esp, costo_tpo, recorrido, recorr_carga, demanda_par, rutas, B, last_choices)
        output[it] = gen
        #print(matriz_impacto)
        candidatos['Probabilidad'] = candidatos['Impacto']/sum(list(candidatos['Impacto']))
        choice = rand.choices (list(candidatos.index), weights = candidatos['Probabilidad'])
        servicio = choice[0]
        estacion = candidatos.loc[choice[0]][0]
        last_choices.append([servicio,estacion])
        #print(servicio, estacion)
        if estacion not in recorrido[servicio]:
            operacion = 'P'
            for i in range(len(recorrido[servicio])):
              index = estaciones_orden.index(estacion)
              if estaciones_orden.index(recorridoh[servicio][i]) > index:
                    recorridoh[servicio].insert(i,estacion)
                    break
            with open('lineas_alt.csv', 'w') as f:
                for key in recorridoh.keys():
                    row = [key, frecuencia[key], 180] 
                    row += recorridoh[key]
                    writer = csv.writer(f,quoting=csv.QUOTE_NONE)
                    if len(key) == 3:        
                        writer.writerow(row)
        else:
            operacion = 'Q'
            recorridoh[servicio].remove(estacion)
            with open('lineas_alt.csv', 'w') as f:
                for key in recorridoh.keys():
                    row = [key, frecuencia[key], 180]
                    row += recorridoh[key]
                    writer = csv.writer(f,quoting=csv.QUOTE_NONE)
                    if len(key) == 3:        
                        writer.writerow(row)
        #print(recorrido[servicio])
        #print('Probando nueva solución')
        choices.append(operacion+servicio+estacion)
        t_viaje_ah, estaciones, Bh = leer(dist=distance_file)
        k, p, d, Ah, Lh, frecuenciah, capacidad, recorridoh, segmentoh, lineas_comunesh, ttvh, tiempo_de_viajeh = crear(Bh, dist = distance_file, lineas = 'lineas_alt.csv', demanda = demand_file)
        frec_l_comh, ttvh, lineas_comunesh = chriquiB (frecuenciah, tiempo_de_viajeh, Lh, ttvh, segmentoh, lineas_comunesh, k, p, d)
        vst_esp, costo_dist, costo_tpo, vel, cap, demanda_par, t_discrh, recorr_cargah, rutash, costo_totalh, costo_servicioh, excesoh, exc_sth, costo_discrh = Costos (Ah,Bh,Lh,pen_ex,k,p,d,ttvh,frec_l_comh,frecuenciah,lineas_comunesh,recorridoh,segmentoh,t_viaje_ah,demandacsv=demand_file)
        cost_diff = costo_totalh - costo
        perc_diff = (cost_diff/costo_total)*100
        variacion.append(perc_diff)
        if costo_totalh > costo:
            kpd = x * perc_diff #tolerancia al empeoramiento
            randomn = rand.uniform(0,1)
            if randomn > math.exp(kpd/t):
                decision = 'Rechazar'
            if randomn <= math.exp(kpd/t):
                with open(line_file, 'w') as f:
                    for key in recorridoh.keys():
                        row = [key,frecuencia[key]]
                        row += recorridoh[key] #puse una h, y en la linea 266 igual
                        writer = csv.writer(f,quoting=csv.QUOTE_NONE)
                        writer.writerow(row)
                df, df_freq, frecuencia = Escribir(df, df_freq, frecuencia,B, recorrido)
                costo = costo_totalh
                recorrido = recorridoh
                exceso = excesoh
                costo_discr = costo_discr_nvo
                decision = 'Aprobar'
        else: 
            #print('El costo disminuyó')
            with open(line_file, 'w') as f:
                for key in recorridoh.keys():
                    row = [key,frecuencia[key]]
                    row += recorridoh[key]
                    writer = csv.writer(f,quoting=csv.QUOTE_NONE)
                    writer.writerow(row)   
            df, df_freq, frecuencia = Escribir (df, df_freq, frecuencia,B, recorrido)
            costo = costo_totalh
            recorrido = recorridoh
            exceso = excesoh
            costo_discr = costo_discrh
            decision = 'Descendió'
        operador = 'Paradas'
      #del recorr_carga
      costo, exceso, rutas, exc_st, demanda_par, vst_esp, costo_dist, costo_tpo, vel, cap, lineas_comunes, recorr_carga, costo_discr = asignar (pen_ex)
      #print(len(recorr_carga['E1N']),len(recorrido['E1N']))
      numest = [[x,len(recorrido[x])] for x in recorrido]        
      print(numest)
      '''
      for i in recorrido:
        if len(i) == 3:    
          print(i, recorrido[i])
          '''
      temp.append(t)
      costos.append(costo) #redundant?
      historial_costos.append(costo)
      '''
      costos_promedio.append(sum(costos)/len(costos))
      if len(costos_promedio) > 100:
        primer_corte = costos_promedio[-10:]
        segundo_corte = costos_promedio[-20:-10]
        avg_1 = sum(primer_corte) / len(primer_corte)
        avg_2 = sum(segundo_corte) / len(segundo_corte)
        if avg_2*0.95 < avg_1:
          
      '''
      excesos.append(exceso)
      costo_componentes.append(costo_discr)
      #print('Temperatura {}. El costo de la solución es {}'.format(t, costo))
      if decision is not 'Descendió':
        print('{:9.0f} {:12} {:13.0f} {:12.2f}% {:10} {:10.0f} {:10.0f} {:10.0f} {:10.0f} {:10.2f} \
        {:10}'.format(t, operador, costo, perc_diff, decision, costo_discr[0],
                                costo_discr[1], costo_discr[2], round(df_freq['Flota asignada'].sum()),
                                math.exp(kpd/t), choices[-1]))
        row = [t, operador, costo, perc_diff, decision, costo_discr[0],
              costo_discr[1], costo_discr[2], round(df_freq['Flota asignada'].sum()),
              math.exp(kpd/t), choices[-1]]
        probs2.append(math.exp(kpd/t))
      else:
        print('{:9.0f} {:12} {:13.0f} {:12.2f}% {:10} {:10.0f} {:10.0f} {:10.0f} {:10.0f} {:10.2f} \
        {:10}'.format(t, operador, costo, perc_diff, decision, costo_discr[0],
                                costo_discr[1], costo_discr[2],
                                round(df_freq['Flota asignada'].sum()),1.00, choices[-1]))
        row = [t, operador, costo, perc_diff, decision, costo_discr[0],
              costo_discr[1], costo_discr[2], round(df_freq['Flota asignada'].sum()),1.00, choices[-1]]
        probs2.append(1)
      rows.append(row)
      flota_asign.append(df_freq['Flota asignada'].sum())
      t = 0.99*t
      it += 1
      frectot = 0
      historial_frecuencias = historial_frecuencias.append(df_freq['Frecuencia'])
      #el siguiente bloque es un criterio de parada: si en el ultimo 10% de 
      #las iteraciones no se ha mejorado mas de un 10% en promedio, se sale.
      if len(historial_costos) > 10:
        desvest = np.std(historial_costos)
      #print('desvest',desvest)
      
    resultados = resultados.append(pd.DataFrame(rows, columns=resultados.columns))
    costo_discr_df = pd.DataFrame(costo_componentes, columns = ['Costo usuarios', 'Costo operador', 'Costo exceso capacidad'])
    costo_discr_df.to_excel('Costos discriminados.xlsx')
    val_derivadas.to_excel('Derivadas.xlsx')
    resultados.to_excel('Run.xlsx')
    probs = pd.DataFrame(probs)
    probs.to_excel('Probabilidades función vecindario.xlsx')
    historial_frecuencias.to_excel('Historial frecuencias.xlsx')
    odf = pd.DataFrame(data=output)
    odf = odf.T
    odf.to_excel('Evolución cromosoma.xlsx')
    with open('Especificaciones.txt', 'w') as f:
      f.write('line_file: {0} \n'.format(line_file))
      f.write('demand_file: {0} \n'.format(demand_file))
      f.write('distance_file {0} \n'.format(distance_file))
      f.write('flota disponible: {0} \n'.format(flota))
      f.write('punto incial: {0} \n'.format(punto_inicial))
    return temp,costos,excesos,variacion,flota_asign,recorrido,recorr_carga,frecuencia

#Ejecución

In [ ]:
"""# Ejecución

Ejecutar desagregadamente
"""
pen_ex = 1
t_viaje_a, estaciones, B = leer(dist=distance_file)
k, p, d, A, L, frecuencia, capacidad, recorrido, segmento, lineas_comunes, ttv, tiempo_de_viaje = crear(B, dist = distance_file, lineas = line_file, demanda = demand_file)
frec_l_com, ttv, lineas_comunes = chriquiB (frecuencia, tiempo_de_viaje, L, ttv, segmento, lineas_comunes, k, p, d)
vst_esp, costo_dist, costo_tpo, vel, cap, demanda_par, t_discr, recorr_carga, rutas, costo_total, costo_servicio, exceso, exc_st, costo_discr = Costos (A,B,L,pen_ex,k,p,d,ttv,frec_l_com,frecuencia,lineas_comunes,recorrido,segmento,t_viaje_a,demandacsv=demand_file)
df, df_freq, frecuencia = create_df (frecuencia, B, recorrido)
costo_total, exceso, rutas, exc_st, demanda_par, vst_esp, costo_dist, costo_tpo, vel, cap, lineas_comunes, recorr_carga, costo_discr = asignar (pen_ex)
temp,costos,excesos, variacion, flota_asign,recorrido,recorr_carga,frecuencia = Sim_annealing_ciclos (df, df_freq, estaciones)


Temperatura  Operación    Costo         Variación   % C. usuarios C. operador Penalización exc. Flota asign. Prob. acept. Movimiento


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:57: RuntimeWarning: divide by zero encountered in double_scalars


a 0  f(a) -449.12998237470936 b 5.0 f(b) inf
a 2.5  f(a) -449.12998237470936 b 5.0 f(b) 535.4434676252906
a 2.5  f(a) -193.87019904137594 b 3.75 f(b) 535.4434676252906
a 3.125  f(a) -193.87019904137594 b 3.75 f(b) 62.84821162529067
a 3.4375  f(a) -82.96630262264307 b 3.75 f(b) 62.84821162529067
a 3.4375  f(a) -15.2627704699471 b 3.59375 f(b) 62.84821162529067
a 3.4375  f(a) -15.2627704699471 b 3.515625 f(b) 22.36414448430321
  Nombre  Frecuencia  Tiempo de ciclo  Flota asignada  Flota mínima
0    67S    6.523438         1.430873        9.334210           6.0
1    67N    6.523438         1.356174        8.846917           6.0
2    E1S    6.523438         1.430873        9.334210           6.0
3    E1N    6.523438         1.356174        8.846917           6.0
[['67S', 41], ['67N', 38], ['E1S', 41], ['E1N', 38]]
  1000000 Frecuencia           19497         2.06% Aprobar         15861       3636          0         36       1.00         {'67S': -1, '67N': -1, 'E1S': -1, 'E1N': -1}
[['67S',

In [ ]:
df, df_freq, frecuencia = create_df (frecuencia, B, recorrido)
df_freq

,Nombre,Frecuencia,Tiempo de ciclo,Flota asignada,Flota mínima
0,67S,10,1.430873,14.308730,6.0
1,67N,10,1.356174,13.561741,6.0
2,E1S,10,1.430873,14.308730,6.0
3,E1N,10,1.356174,13.561741,6.0


In [ ]:
recorrido['E1S']

['55334',
 '60505',
 '55335',
 '55047',
 '55059',
 '55079',
 '50733',
 '50815',
 '50880',
 '50932',
 '51001',
 '51079',
 '51143',
 '51224',
 '51277',
 '51360',
 '51394',
 '51471',
 '51473',
 '51519',
 '51557',
 '51605',
 '51632',
 '54364',
 '51745',
 '51839',
 '51945',
 '52047',
 '52109',
 '52159',
 '52240',
 '52349',
 '52432',
 '52522',
 '52628',
 '53826',
 '61046',
 '52865',
 '52952',
 '53053',
 '53036']

In [ ]:
recorr_carga['XSN']

{('50732', '55057'): 3.6655263157894735,
 ('50931', '50732'): 3.6655263157894735,
 ('51080', '50931'): 8.438684210526315,
 ('51392', '54180'): 6.776842105263157,
 ('51517', '51392'): 8.726842105263158,
 ('51606', '51517'): 8.726842105263158,
 ('51943', '51606'): 22.777894736842104,
 ('52107', '51943'): 19.342894736842105,
 ('52347', '52107'): 18.16657894736842,
 ('52520', '52347'): 16.987105263157893,
 ('52815', '52520'): 14.366052631578945,
 ('53036', '52815'): 11.13236842105263,
 ('54180', '51080'): 6.776842105263157,
 ('55057', '55333'): 6.563684210526315,
 ('55333', '55334'): 2.898157894736842}

In [ ]:
tc = get_loads(B,recorr_carga,frecuencia,recorrido,all_stop_ida, all_stop_reg)

In [ ]:
rows = []
for par in demanda_par:
  if par[0][:-1] != par[1][:-1]:
    if len(lineas_comunes[par]) == 2:
      for lc in lineas_comunes[par]:
        tt1, tt2 = segmento[par][lc][0],segmento[par][lc][0]
      rows.append([par[0],par[1],demanda_par[par],tt1,tt2,lineas_comunes[par]])
    elif len(lineas_comunes[par]) == 1:
      for lc in lineas_comunes[par]:
        tt1 = segmento[par][lc][0]
      rows.append([par[0],par[1],demanda_par[par],tt1,0,lineas_comunes[par]])


In [ ]:
desglose = pd.DataFrame(rows,columns=['origen','destino','demanda','tt67','tt467','lc'])

In [ ]:
desglose.to_excel('desglose.xlsx')

In [ ]:
recorrido['E1N']

['53036', '52815', '52651', '52347', '51606', '50879', '55334']

In [ ]:
|vst_veh = 820
derivada = {}
for l in frecuencia:
  if len(l) == 3: 
    #derivadas de la función objetivo para conocer su cambio frente a cambios en f
    #derivada calculada en página 93 de mi cuaderno
    #derivada de [costos_operador,tiempo_espera, penalidad, tiempo_viaje], suma
    #derivada tiempo_viaje despreciable
    derivada[l] = [[0,0,0,0], 0]
    #averiguo el index de cada servicio
    index = df_freq.index[df_freq['Nombre'] == l].tolist()[0]
    tc = df_freq.loc[index]['Tiempo de ciclo']
    #distancia y tiempo que se recorren en una hora
    d, tpox = vel, 1
    #derivada del costo del operador
    derivada[l][0][0] = tc*(costo_tpo*tc+costo_dist*vel*tc)
    #derivada del tiempo de espera 
    ies, ivi = 0, 0
    for par in demanda_par:
      #experimentar con secciones de ruta por cada par
      if par[0][:-1] != par[1][:-1] and l in lineas_comunes[par]:
        F = 0
        tf = 0
        try:
          lc = lineas_comunes[par]
          for lca in lc:
            F = F + frecuencia[lca]
            tf = tf + segmento[par][lca][0]*frecuencia[lca]
        except:
          pass
        ies = ies + ((demanda_par[par]*(frecuencia[l]/F)*vst_esp)*((-k)/(F**2)))
        num1 = float(segmento[par][l][0])*F
        ivi = ivi + (demanda_par[par]*(frecuencia[l]/F)*vst_veh*((num1-tf)/F**2))
    derivada[l][0][1] = derivada[l][0][1] + ies #avancé esto un tab
    derivada[l][0][3] = derivada[l][0][3] + ivi
    if exc_st[l] > 0:
      derivada[l][0][2] = -cap*pen_ex
    derivada[l][1] = sum(derivada[l][0])

#print(derivada)
C = nx.MultiDiGraph()
for l in derivada:
  index = df_freq.loc[df_freq['Nombre'] == l].index[0]
  if df_freq.iloc[index]['Frecuencia'] <= 1:
    #Los servicios que tienen frecuencia menor a 1, solo pueden aumentar
    #su frecuencia.
    C.add_edge(recorrido[l][0], recorrido[l][-1], capacity = 1, weight = derivada[l][1], key = l+'+')
  else:
    C.add_edge(recorrido[l][0], recorrido[l][-1], capacity = 1, weight = derivada[l][1], key = l+'+')
    C.add_edge(recorrido[l][-1], recorrido[l][0], capacity = 1, weight = -derivada[l][1], key = l+'-')
delta, solucion = nx.algorithms.flow.network_simplex(C, capacity = 'capacity', weight = 'weight')

In [ ]:
for i in frecuencia:
  if len(i) == 3: 
    print(frecuencia[i])

In [ ]:
recorr_carga

In [ ]:
delta, solucion

(-987.0925618000153,
 {'53036': {'55334': {'67N+': 1, '67S-': 0, 'E1N+': 1, 'E1S-': 0}},
  '55334': {'53036': {'67N-': 0, '67S+': 1, 'E1N-': 0, 'E1S+': 1}}})

In [ ]:
#df_freq['Frecuencia'] = [100,1,1,1]
#df, df_freq, frecuencia = Escribir(df, df_freq, frecuencia,B, recorrido)
#vst_esp, costo_dist, costo_tpo, vel, cap, demanda_par, t_discr, recorr_carga, rutas, costo_total, costo_servicio, exceso, exc_st, costo_discr = Costos (A,B,L,pen_ex,k,p,d,ttv,frec_l_com,frecuencia,lineas_comunes,recorrido,segmento,t_viaje_a,demandacsv=demand_file)
pen_ex = 1
vst_veh = 7 #CLP/h
vst_esp = 2*vst_veh
vst_trb = 4*vst_veh
vel = 20 #km/h
costo_dist = 4 #CLP/km
costo_tpo = 20 #CLP/h
#k, p, d, A, L, frecuencia, capacidad, recorrido, segmento, lineas_comunes, ttv, tiempo_de_viaje = crear(B, dist = distance_file, lineas = line_file, demanda = demand_file)
#df, df_freq, frecuencia = create_df (frecuencia, B, recorrido)
#costo_total, exceso, rutas, exc_st, demanda_par, vst_esp, costo_dist, costo_tpo, vel, cap, lineas_comunes, recorr_carga, costo_discr = asignar (pen_ex)
t_viaje_a, estaciones, B = leer(dist=distance_file)
k, p, d, A, L, frecuencia, capacidad, recorrido, segmento, lineas_comunes, ttv, tiempo_de_viaje = crear(B, dist = distance_file, lineas = line_file, demanda = demand_file)
frec_l_com, ttv, lineas_comunes = chriquiB (frecuencia, tiempo_de_viaje, L, ttv, segmento, lineas_comunes, k, p, d)
vst_esp, costo_dist, costo_tpo, vel, cap, demanda_par, t_discr, recorr_carga, rutas, costo_total, costo_servicio, exceso, exc_st, costo_discr = Costos (A,B,L,pen_ex,k,p,d,ttv,frec_l_com,frecuencia,lineas_comunes,recorrido,segmento,t_viaje_a,demandacsv=demand_file)
print(costo_total)

19139.24264784836


In [ ]:
len(recorr_carga['E1N'])

37

In [ ]:
for serv in recorr_carga:
  for par in recorr_carga[serv]:
    print("Service {} between {} takes {} pass.".format(serv,par,recorr_carga[serv][par]/float(frecuencia[serv])))

In [ ]:
for i in demanda_par:
  if '52922' in i:
    if i[0] in recorrido['E1S'] or i[1] in recorrido['E1S']:
      print(i, demanda_par[i])

('52922', '55334') 31.79


In [ ]:
df_rc = pd.DataFrame(data=recorr_carga)
df_rc.to_excel('cargas.xlsx')

In [ ]:
recorrido['XSS']

['55334',
 '60505',
 '55059',
 '50733',
 '50932',
 '51079',
 '51224',
 '51394',
 '51519',
 '51605',
 '51745',
 '51945',
 '52109',
 '52349',
 '52522',
 '61046',
 '52952',
 '53036']

In [ ]:
recorr_carga['XSS']

KeyError: ignored

In [ ]:
costo_total, exceso, rutas, exc_st, demanda_par, vst_esp, costo_dist, costo_tpo, vel, cap, lineas_comunes, recorr_carga, costo_discr = asignar (pen_ex)
dfd = pd.DataFrame(columns=['Suma Frecuencia', 'Costo', 'Costo Viaje', 'Costo Op', 'Costo Pen'])
'''
for i in derivada:
  print(delta)
  print(solucion)
  print('Original', costo_total, costo_discr)

'''
for o in range(0,40):
  paso = o*2
  df_freq.at[0, 'Frecuencia'] -= paso
  df_freq.at[1, 'Frecuencia'] -= paso
  df_freq.at[2, 'Frecuencia'] += paso
  df_freq.at[3, 'Frecuencia'] += paso
  df_freq['Flota asignada'] = df_freq['Frecuencia']*df_freq['Tiempo de ciclo']
  df, df_freq, frecuencia = Escribir (df, df_freq, frecuencia,B, recorrido)
  vst_esp, costo_dist, costo_tpo, vel, cap, demanda_par, t_discr, recorr_carga, rutas, costo_total, costo_servicio, exceso, exc_st, costo_discr = Costos (A,B,L,pen_ex,k,p,d,ttv,frec_l_com,frecuencia,lineas_comunes,recorrido,segmento,t_viaje_a,demandacsv=demand_file)
  df, df_freq, frecuencia = create_df (frecuencia, B, recorrido)
  costo_total, exceso, rutas, exc_st, demanda_par, vst_esp, costo_dist, costo_tpo, vel, cap, lineas_comunes, recorr_carga, costo_discr = asignar (pen_ex)
  row = [sum(df_freq['Flota asignada']), costo_total, costo_discr[0], costo_discr[1], costo_discr[2]]
  print(sum(df_freq['Flota asignada']), costo_total)
  #print(df_freq)
  dfd = dfd.append([row])
  df_freq.at[0, 'Frecuencia'] += paso
  df_freq.at[1, 'Frecuencia'] += paso
  df_freq.at[2, 'Frecuencia'] -= paso
  df_freq.at[3, 'Frecuencia'] -= paso
  df_freq['Flota asignada'] = df_freq['Frecuencia']*df_freq['Tiempo de ciclo']
  df, df_freq, frecuencia = Escribir (df, df_freq, frecuencia,t_viaje_a, recorrido)
  vst_esp, costo_dist, costo_tpo, vel, cap, demanda_par, t_discr, recorr_carga, rutas, costo_total, costo_servicio, exceso, exc_st, costo_discr = Costos (A,B,L,pen_ex,k,p,d,ttv,frec_l_com,frecuencia,lineas_comunes,recorrido,segmento,t_viaje_a,demandacsv=demand_file)
  df, df_freq, frecuencia = create_df (frecuencia, B, recorrido)
  costo_total, exceso, rutas, exc_st, demanda_par, vst_esp, costo_dist, costo_tpo, vel, cap, lineas_comunes, recorr_carga, costo_discr = asignar (pen_ex)
